https://womantalks.ru/forum/65-puteshestviya/

In [1]:
# pip install bs

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm

In [2]:
df = pd.DataFrame(columns=['text'])
df

,text


In [3]:
url = 'https://womantalks.ru/forum/65-puteshestviya/'

In [4]:
response = requests.get(url)
print(response)

<Response [200]>


In [5]:
html = response.text

In [6]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [7]:
import re

In [8]:
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    return sorted(list(set(links)))
get_links(get_html(url), url)

['https://womantalks.ru/forum/65-puteshestviya/?page=1',
 'https://womantalks.ru/forum/65-puteshestviya/?page=2']

In [43]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_large ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if 'topic' in q['href']:
                links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('strong', 'itemprop'=="name")
    users = []
    for td in tds:
        if 'Изменено' not in td.text:
            users.append(td.text)
    print(users)


    # time
    tds = soup.find_all('p', {'class':"ipsType_reset"})
    times = []
    for td in tds:
        if 'Опубликовано' in td.text:
            times.append(td.text)
    print(times)


    # text
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    tex = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find_all('p')
            tex.append(p[-1].text)
        else:
            tex.append(td.text)
    print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    temp = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find('div', {'class':"ipsQuote_contents"})
            if p is not None:
                temp.append(p.text)
        else:
            temp.append('')
    print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Travel'] * len(users)
            })          
    return data

def page_content(all_links, table):
#     for link in tqdm(all_links):
#     links = get_links(get_html(all_links), all_links)
    i = 0
    for l in tqdm(all_links):
        all_links_one_theme = get_all_links_one_theme(get_html(l))
        if i == 0:
            all_links_one_theme.append('https://womantalks.ru/topic/1517-turizm/')
            all_links_one_theme.append('https://womantalks.ru/topic/102-veloturizm/')
            i=1
        
        for t in tqdm(all_links_one_theme):
            print(t)
            page = get_links(get_html(t), t)
            for p in page:
#                 print(p)
                data = get_page_data(p)
                table.append(data)

table = []
    
url = 'https://womantalks.ru/forum/65-puteshestviya/'

all_links = get_links(get_html(url), url)
print(all_links)
page_content(all_links, table)

['https://womantalks.ru/forum/65-puteshestviya/?page=1', 'https://womantalks.ru/forum/65-puteshestviya/?page=2']


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

https://womantalks.ru/topic/6913-gde-predpochitaete-puteshestvovat/
['\nГость', '\nJasmiine', '\nGolubka']
['\nОпубликовано: 21 мар 2019\n', '\nОпубликовано: 1 июн 2019\n', '\nОпубликовано: 30 янв 2021\n']
['\n\n\tПоделитесь, пожалуйста кто где путешествует ?\xa0\n\tЕщё интересно все-таки сравнить сервис в России и за границей. Поделитесь своими историями из жизни путешественника будет интересно!\xa0\n\n', '\n\n\tПока что нашему сервису очень далеко от, скажем, европейского, к сожалению.\xa0\n\n', '\n\n\tВезде, где есть что посмотреть)\n\n']
['', '', '']
https://womantalks.ru/topic/1246-kuda-poedete-v-otpusk/
['\nДарина', '\nМиленькая)', '\nЛаванда', '\nLizz', '\nDeneb', '\nФиона', '\nМандаринка', '\nСердцеедка', '\nСтервочка)', '\nИришка', '\nЛиана', '\nблонда2', '\nfilata', '\nМанка', '\nTalita', '\nМанка', '\ntilinka', '\nRealmccoy', '\nТирион', '\nNoKaty', '\nAugustina', '\nMexika', '\nКарина Морозова', '\nМирка', '\ngomiss', '\nДжули', '\nАдам Малхиев', '\nYllianna', '\nМирная-зеф

['\nМарта', '\nXyhuvupu', '\nOlgaOlga_5', '\nTata84', '\nLisaAliska', '\nСаленька', '\nМаршкина', '\nAugustina', '\nГость Маргарита', '\nСветлана89', '\nВероника В', '\nSveta812', '\nОксана Ларюкова', '\nBonita', '\nMariana.Ri', '\nKristinWow', '\nAugustina', '\nKarinochka', '\nvipmani', '\nГость', '\nГость', '\nsasha_kyz', '\nKate_nochek', '\nWomanInRock', '\nwomantalks.ru', '\nDelta', '\nvalentina1973', '\nВикарус', '\nKonst', '\nolga_inavova', '\nAlexxys', '\nppolik', '\nКюшечкаН', '\nNikaWeronika', '\nLLeennuusya']
['\nОпубликовано: 13 мар 2017\n', '\nОпубликовано: 27 мар 2017\n', '\nОпубликовано: 27 мар 2017\n', '\nОпубликовано: 27 мар 2017\n', '\nОпубликовано: 29 мар 2017\n', '\nОпубликовано: 2 апр 2017\n', '\nОпубликовано: 2 апр 2017\n', '\nОпубликовано: 26 апр 2017\n', '\nОпубликовано: 9 май 2017\n', '\nОпубликовано: 10 май 2017\n', '\nОпубликовано: 11 май 2017\n', '\nОпубликовано: 11 май 2017\n', '\nОпубликовано: 12 май 2017\n', '\nОпубликовано: 12 май 2017\n', '\nОпубликовано

...

['\nГость', '\ndjumba', '\nВероника В', '\nTwiksi', '\nvipmani', '\nресничка', '\nТараканчик', '\nLureks', '\nRealmccoy', '\nLorena', '\nAugustina', '\nLorena', '\nanmelnik93', '\nОльга 77']
['\nОпубликовано: 5 апр 2017\n', '\nОпубликовано: 13 апр 2017\n', '\nОпубликовано: 6 июл 2017\n', '\nОпубликовано: 6 июл 2017\n', '\nОпубликовано: 11 июл 2017\n', '\nОпубликовано: 27 июл 2017\n', '\nОпубликовано: 16 авг 2017\n', '\nОпубликовано: 26 авг 2017\n', '\nОпубликовано: 28 янв 2018\n', '\nОпубликовано: 15 май 2018\n', '\nОпубликовано: 16 май 2018\n', '\nОпубликовано: 28 май 2018\n\t\t\t\n\t\t\t\t(изменено)\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t', '\nОпубликовано: 11 июн 2018\n', '\nОпубликовано: 17 июн 2018\n']
['\n\n\tБаза отдыха Дубки предлагает прекрасный вариант отдыха в старорусских традициях\n\n', '\n', '\n\n\tНам нравится отдыхать в Сочи. когда отпуск не планируется в конкретную страну, не строятся никакие планы, мы всегда знаем, что может полететь на черное море в Сочи. Вот и сейчас тут живе

['\nDiana', '\nGalo4ka', 'Еще одни насущный вопрос', '\nTanana', '\nСнежинка', '\nЗиноида', '\nlucky.starsky', '\nDeneb', '\nXyhuvupu', '\nZidogaj', '\nAlina90', '\nprostomary', '\nTalita', '\nVioletaYa', '\nagafonia', '\nprostomary', '\nСапфировая', '\nMarinka_ya', '\nAleka', '\nAugustina', '\nXuos', '\nMandarinka']
['\nОпубликовано: 24 июл 2015\n', '\nОпубликовано: 25 июл 2015\n', '\nОпубликовано: 9 сен 2015\n', '\nОпубликовано: 9 сен 2015\n', '\nОпубликовано: 21 окт 2015\n', '\nОпубликовано: 29 окт 2015\n', '\nОпубликовано: 27 фев 2016\n', '\nОпубликовано: 12 июл 2016\n', '\nОпубликовано: 12 июл 2016\n', '\nОпубликовано: 4 авг 2016\n', '\nОпубликовано: 15 авг 2016\n', '\nОпубликовано: 16 авг 2016\n', '\nОпубликовано: 16 авг 2016\n', '\nОпубликовано: 17 авг 2016\n', '\nОпубликовано: 20 авг 2016\n', '\nОпубликовано: 22 авг 2016\n', '\nОпубликовано: 22 сен 2016\n', '\nОпубликовано: 27 янв 2017\n', '\nОпубликовано: 27 янв 2017\n', '\nОпубликовано: 27 янв 2017\n', '\nОпубликовано: 15 июл

  0%|          | 0/9 [00:00<?, ?it/s]

https://womantalks.ru/topic/3718-chto-znachit-dlya-vas-puteshestvovat/
['\nОльга1329', '\nAugustina', '\nВероника В', '\nЛеночек', '\nKarinochka', '\nAugustina', '\nLizaCute']
['\nОпубликовано: 26 апр 2017\n', '\nОпубликовано: 27 апр 2017\n', '\nОпубликовано: 27 май 2017\n', '\nОпубликовано: 30 май 2017\n', '\nОпубликовано: 10 июл 2017\n', '\nОпубликовано: 16 окт 2017\n', '\nОпубликовано: 16 окт 2017\n']
['\n\n\tРасскажите свое понимание этого слова?\n\n', '\n\n\tдля меня это расширение кругозора, незабываемые эмоции, практики в общении и в языках)\n\n', '\n\n\tО да,в первую очередь новые эмоции! Мы так хотим поехать в Тай, когда смотрю фотографии, просто хочется туда телепортироваться) Кто-то ездил туда с ребенком, мне как-то страшновато(\n\n', '\n\n\tДля меня это смена остановки и конечно же отдых от суеты и работы) Ну и конечно возможность наполниться силами на следующий период до отпуска) а с ребенком в тай конечно можно ехать, там хорошо, фрукты, природа) только вот подстрахуйтесь

['\nГость Галочка', '\nRealmccoy', '\nTalita', '\nТирион', '\ngrogozhan', '\nTonya7', '\nAugustina']
['\nОпубликовано: 1 авг 2016\n', '\nОпубликовано: 1 авг 2016\n', '\nОпубликовано: 1 авг 2016\n', '\nОпубликовано: 24 авг 2016\n', '\nОпубликовано: 30 авг 2016\n', '\nОпубликовано: 11 сен 2016\n', '\nОпубликовано: 10 май 2017\n']
['\n\n\tДавайте делиться местами отдыха и отзывами о курортах.\n\n', '\n\n\tМогу посоветовать курорт в Краснодарском крае возле Ейска, так сказать по горячим следам - только вернулись оттуда. Отдыхали на косе Долгой в 40 км от Ейска на базе "Казачий берег". Всем очень понравилось - и детям, и взрослым. Ездили всей семьёй - мы с мужем, дети и бабушка. Чудное местечко, нам кума рекомендовала его. Море чистое с хорошим дном, неглубокое - для деток самое то, на берегу мелкий ракушечник. Очень понравилось питание на базе (шведский стол), уютные номера, ухоженная территория, а уж развлечений сколько - всего и не перепробовали наверное.\xa0\n\n\n\n\n', '\n\n\tПоселок К

In [44]:
pd.DataFrame(table).to_csv('Travel_woman_talks.csv', index=False)

In [2]:
D = pd.read_csv('Travel_woman_talks.csv')
D

,user,time,text,quote,topic
0,"['\nГость', '\nJasmiine', '\nGolubka']","['\nОпубликовано: 21 мар 2019\n', '\nОпубликов...","['\n\n\tПоделитесь, пожалуйста кто где путешес...","['', '', '']","['Travel', 'Travel', 'Travel']"
1,"['\nДарина', '\nМиленькая)', '\nЛаванда', '\nL...","['\nОпубликовано: 18 апр 2016\n', '\nОпубликов...",['\n\n\tКуда вы хотите поехать в отпуск? Я бы ...,"['', '', '', '', '\n\n\t\t\tМы пока еще не выб...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
2,"['\nМарта', '\nXyhuvupu', '\nOlgaOlga_5', '\nT...","['\nОпубликовано: 13 мар 2017\n', '\nОпубликов...",['\n\n\tНам как вариант предложили купить недв...,"['', '', '\n\n\t\t\tа я по стране хочу поездит...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
3,"['\nEkaterinaN', '\nLolaa', '\nAlivia', '\nКюш...","['\nОпубликовано: 23 апр 2019\n', '\nОпубликов...","['\n\n\tГреция по дороже, но гораздо лучше. Бы...","['', '', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
4,"['\nGolubka', '\nbaranets20-20', '\nGolubka', ...","['\nОпубликовано: 30 янв 2021\n', '\nОпубликов...","['\n\n\tЧто лучше, ехать со своей карточкой ил...","['', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel']"
5,"['\nAlexa', '\nNissa', '\nМикра', '\nAlexa', '...","['\nОпубликовано: 2 мар 2016\n', '\nОпубликова...",['\n\n\tЧасто встречаю приверженников самостоя...,"['', '', '\n\n\t\t\tЧасто встречаю приверженни...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
6,"['\nikari', '\ncorenad', '\nKetaryKa']","['\nОпубликовано: 3 авг 2020\n', '\nОпубликова...","['\n\n\t\xa0Как сейчас дела с развлечениями, ч...","['', '', '']","['Travel', 'Travel', 'Travel']"
7,"['\nKatosha', '\nАнтонина89', '\nГалина Марков...","['\nОпубликовано: 12 фев 2020\n', '\nОпубликов...",['\n\n\tДавайте поговорим об Израиле. Кто был?...,"['', '', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
8,"['\nБьянка-Янка', '\nМайская', '\nAleksandra11...","['\nОпубликовано: 28 май 2016\n', '\nОпубликов...",['\n\n\tКак предпочитаете отдыхать - валяться ...,"['', '', '', '', '', '', '', '', '', '', '', '...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
9,['\nVenezian'],['\nОпубликовано: 17 дек 2019\n'],"['\n\n\tВсем привет, очередной вопрос про пода...",[''],['Travel']


In [46]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 415


In [3]:
print(D.topic[1].split("',"))
print(len(D.topic[1].split("',")))

["['Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel", " 'Travel']"]
35


In [4]:
user = []
for i in range(len(D)):
    user.extend(D.user[i].split("',"))

In [15]:
user.remove(" 'Еще одни насущный вопрос")

In [16]:
len(user)

378

In [6]:
time = []
for i in range(len(D)):
    time.extend(D.time[i].split("',"))

In [7]:
len(time)

378

In [8]:
text = []
for i in range(len(D)):
    text.extend(D.text[i].split("\',"))

In [9]:
len(text)

378

In [10]:
quote = []
for i in range(len(D)):
    quote.extend(D.quote[i].split("',"))

In [11]:
len(quote)

378

In [12]:
topic = []
for i in range(len(D)):
    topic.extend(D.topic[i].split("',"))

In [17]:
topic = topic[:-1]

In [18]:
len(topic)

378

In [19]:
table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
table

,text,quote,users,times,topic


In [20]:
table.loc[:, 'users'] = user
table.loc[:, 'times'] = time
table.loc[:, 'text'] = text
table.loc[:, 'quote'] = quote
table.loc[:, 'topic'] = topic

In [21]:
table

,text,quote,users,times,topic
0,"['\n\n\tПоделитесь, пожалуйста кто где путешес...",[',['\nГость,['\nОпубликовано: 21 мар 2019\n,['Travel
1,'\n\n\tПока что нашему сервису очень далеко о...,','\nJasmiine,'\nОпубликовано: 1 июн 2019\n,'Travel
2,"'\n\n\tВезде, где есть что посмотреть)\n\n']",''],'\nGolubka'],'\nОпубликовано: 30 янв 2021\n'],'Travel']
3,['\n\n\tКуда вы хотите поехать в отпуск? Я бы ...,[',['\nДарина,['\nОпубликовано: 18 апр 2016\n,['Travel
4,"'\n\n\tГоворят, что на Кипр дешевле и на Крит...",','\nМиленькая),'\nОпубликовано: 18 апр 2016\n,'Travel
...,...,...,...,...,...
373,"'\n\n\tКлассно там, судя по всему... жаль тол...",','\nprostomary,'\nОпубликовано: 7 сен 2016\n,'Travel
374,"'\n\n\tЕсли именно в России хотите, тогда тол...",','\nLolly_Polly,'\nОпубликовано: 8 сен 2016\n,'Travel
375,"'\n\n\tДа мне кажется, недорого сейчас не най...",''],'\nprostomary'],'\nОпубликовано: 13 сен 2016\n'],'Travel
376,['\n\n\tВсем добрый день! Кто отдыхал в Подмос...,[',['\nMulya,['\nОпубликовано: 5 июл 2016\n,'Travel']


In [22]:
import re

In [23]:
table.loc[:, 'text'] = table.text.apply(lambda x: re.sub(r"[\[\]\'\\n\\t\\r\\xa0]", '', str(x)))

In [24]:
table.loc[:, 'quote'] = table.quote.apply(lambda x: re.sub(r"[\[\]\'\\n\\n\\t\\r\\xa0]", '', str(x)))

In [25]:
table.loc[:, 'quote'] = table.quote.apply(lambda x: re.sub(r"^ *$", '', str(x)))

In [26]:
table.loc[:, 'users'] = table.users.apply(lambda x: re.sub(r"[\[\]\'\\n\\n\\t\\r\\xa0]", '', str(x)))

In [27]:
table.loc[:, 'topic'] = table.topic.apply(lambda x: re.sub(r"[\[\]\'\n\t\r\xa0]", '', str(x)))

In [28]:
table.loc[:, 'times'] = table.times.apply(lambda x: re.sub(r"[\[\]\'\n\t\r\xa0\\n]", '', str(x)))

In [29]:
table.loc[:, 'times'] = table.times.apply(lambda x: re.sub(r'Опубликовано: ', '', str(x)))

In [30]:
table

,text,quote,users,times,topic
0,"Поделитесь, пожалуйста кто где путешествует ?Е...",,Гость,21 мар 2019,Travel
1,"Пока что нашему сервису очень далеко от, скаж...",,Jsmiie,1 июн 2019,Travel
2,"Везде, где есть что посмотреть)",,Golubk,30 янв 2021,Travel
3,Куда вы хотите поехать в отпуск? Я бы с удовол...,,Дарина,18 апр 2016,Travel
4,"Говорят, что на Кипр дешевле и на Крит тоже. ...",,Миленькая),18 апр 2016,Travel
...,...,...,...,...,...
373,"Классно там, судя по всему... жаль только на ...",,posomy,7 сен 2016,Travel
374,"Если именно в России хотите, тогда только Соч...",,Lolly_Polly,8 сен 2016,Travel
375,"Да мне кажется, недорого сейчас не найдешь ни...",,posomy,13 сен 2016,Travel
376,"Всем добрый день! Кто отдыхал в Подмосковье, п...",,Muly,5 июл 2016,Travel


In [31]:
table.to_csv('Travel_1.csv', index=False)

In [32]:
D = pd.read_csv('Travel_1.csv')
D

,text,quote,users,times,topic
0,"Поделитесь, пожалуйста кто где путешествует ?Е...",NaN,Гость,21 мар 2019,Travel
1,"Пока что нашему сервису очень далеко от, скаж...",NaN,Jsmiie,1 июн 2019,Travel
2,"Везде, где есть что посмотреть)",NaN,Golubk,30 янв 2021,Travel
3,Куда вы хотите поехать в отпуск? Я бы с удовол...,NaN,Дарина,18 апр 2016,Travel
4,"Говорят, что на Кипр дешевле и на Крит тоже. ...",NaN,Миленькая),18 апр 2016,Travel
...,...,...,...,...,...
373,"Классно там, судя по всему... жаль только на ...",NaN,posomy,7 сен 2016,Travel
374,"Если именно в России хотите, тогда только Соч...",NaN,Lolly_Polly,8 сен 2016,Travel
375,"Да мне кажется, недорого сейчас не найдешь ни...",NaN,posomy,13 сен 2016,Travel
376,"Всем добрый день! Кто отдыхал в Подмосковье, п...",NaN,Muly,5 июл 2016,Travel
